In [5]:
import cv2
import numpy as np
import os

# --------------------------
# CONFIG
# --------------------------
input_folder = r"D:\projects\neuro\20x_bf_05_all_patches\3"
output_root = r"D:\projects\neuro\tissue_mask_debug_20xbf05all_3"
os.makedirs(output_root, exist_ok=True)

valid_exts = (".png", ".jpg", ".jpeg", ".bmp")

# --------------------------
# Tissue Masking (STEP-WISE)
# --------------------------
def tissue_mask_debug(image, save_dir):
    os.makedirs(save_dir, exist_ok=True)

    # 1. Save original
    cv2.imwrite(os.path.join(save_dir, "01_original.png"), image)

    # 2. Convert to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    cv2.imwrite(os.path.join(save_dir, "02_hsv.png"), hsv)

    H, S, V = cv2.split(hsv)

    # 3. Save channels
    cv2.imwrite(os.path.join(save_dir, "03_H.png"), H)
    cv2.imwrite(os.path.join(save_dir, "04_S.png"), S)
    cv2.imwrite(os.path.join(save_dir, "05_V.png"), V)

    # 4. Saturation mask
    sat_mask = cv2.inRange(S, 15, 255)
    cv2.imwrite(os.path.join(save_dir, "06_sat_mask.png"), sat_mask)

    # 5. Brightness mask (exclude very bright background)
    not_too_bright = cv2.inRange(V, 0, 250)
    cv2.imwrite(os.path.join(save_dir, "07_not_too_bright.png"), not_too_bright)

    # 6. Combined mask
    combined = cv2.bitwise_and(sat_mask, not_too_bright)
    cv2.imwrite(os.path.join(save_dir, "08_combined_mask.png"), combined)

    # 7. Morphological kernel
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))

    # 8. Morph OPEN
    opened = cv2.morphologyEx(combined, cv2.MORPH_OPEN, k, iterations=1)
    cv2.imwrite(os.path.join(save_dir, "09_opened.png"), opened)

    # 9. Morph CLOSE
    closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, k, iterations=2)
    cv2.imwrite(os.path.join(save_dir, "10_closed_final_mask.png"), closed)

    return closed

# --------------------------
# Process folder
# --------------------------
images = [
    f for f in os.listdir(input_folder)
    if f.lower().endswith(valid_exts)
]

print(f"Found {len(images)} images")

for img_name in images:
    print(f"Processing {img_name}")

    img_path = os.path.join(input_folder, img_name)
    image = cv2.imread(img_path)

    if image is None:
        print("  Could not read image, skipping")
        continue

    img_name_wo_ext = os.path.splitext(img_name)[0]
    save_dir = os.path.join(output_root, img_name_wo_ext)

    tissue_mask_debug(image, save_dir)

print("\n DONE! Tissue mask debug images saved.")
#hsc,h,s,v,not bright, not saturated, combined, open ,close images just for the 3rd fascicle of 20x_bf_05_all_patches 

Found 42 images
Processing patch_00000.png
Processing patch_00001.png
Processing patch_00002.png
Processing patch_00003.png
Processing patch_00004.png
Processing patch_00005.png
Processing patch_00006.png
Processing patch_00007.png
Processing patch_00008.png
Processing patch_00009.png
Processing patch_00010.png
Processing patch_00011.png
Processing patch_00012.png
Processing patch_00013.png
Processing patch_00014.png
Processing patch_00015.png
Processing patch_00016.png
Processing patch_00017.png
Processing patch_00018.png
Processing patch_00019.png
Processing patch_00020.png
Processing patch_00021.png
Processing patch_00022.png
Processing patch_00023.png
Processing patch_00024.png
Processing patch_00025.png
Processing patch_00026.png
Processing patch_00027.png
Processing patch_00028.png
Processing patch_00029.png
Processing patch_00030.png
Processing patch_00031.png
Processing patch_00032.png
Processing patch_00033.png
Processing patch_00034.png
Processing patch_00035.png
Processing p

In [1]:
import cv2
import numpy as np
import os

# ==================================================
# CONFIG (CHANGE ONLY THESE PATHS)
# ==================================================
PATCH_ROOT = r"D:\projects\neuro\20x_bf_05_all_patches"
FULL_IMAGE_ROOT = r"D:\projects\neuro\all fascicles\fascicle images"

PATCH_OUTPUT_ROOT = r"D:\projects\neuro\tissue_mask_debug_patches"
FULL_OUTPUT_ROOT = r"D:\projects\neuro\tissue_mask_debug_full"

VALID_EXTS = (".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff")

# ==================================================
# TISSUE MASK DEBUG (UNCHANGED LOGIC)
# ==================================================
def tissue_mask_debug(image, save_dir):
    os.makedirs(save_dir, exist_ok=True)

    cv2.imwrite(os.path.join(save_dir, "01_original.png"), image)

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    cv2.imwrite(os.path.join(save_dir, "02_hsv.png"), hsv)

    H, S, V = cv2.split(hsv)
    cv2.imwrite(os.path.join(save_dir, "03_H.png"), H)
    cv2.imwrite(os.path.join(save_dir, "04_S.png"), S)
    cv2.imwrite(os.path.join(save_dir, "05_V.png"), V)

    sat_mask = cv2.inRange(S, 15, 255)
    cv2.imwrite(os.path.join(save_dir, "06_sat_mask.png"), sat_mask)

    not_too_bight = cv2.inRange(V, 0, 250)
    cv2.imwrite(os.path.join(save_dir, "07_not_too_bright.png"), not_too_bight)

    combined = cv2.bitwise_and(sat_mask, not_too_bight)
    cv2.imwrite(os.path.join(save_dir, "08_combined_mask.png"), combined)

    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    opened = cv2.morphologyEx(combined, cv2.MORPH_OPEN, k, iterations=1)
    cv2.imwrite(os.path.join(save_dir, "09_opened.png"), opened)

    closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, k, iterations=2)
    cv2.imwrite(os.path.join(save_dir, "10_closed_final_mask.png"), closed)

    return closed

# ==================================================
# OVERLAP VISUALIZATION
# ==================================================
def visualize_overlap(image, tissue_mask, save_dir):
    os.makedirs(save_dir, exist_ok=True)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    _, fg = cv2.threshold(
        gray, 0, 255,
        cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU
    )

    contours, _ = cv2.findContours(
        fg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    # Combined masks
    all_contours_mask = np.zeros(tissue_mask.shape, dtype=np.uint8)
    all_overlap_mask = np.zeros(tissue_mask.shape, dtype=np.uint8)

    for c in contours:
        c_mask = np.zeros(tissue_mask.shape, dtype=np.uint8)
        cv2.drawContours(c_mask, [c], -1, 255, -1)

        overlap = cv2.bitwise_and(c_mask, tissue_mask)

        # Accumulate
        all_contours_mask = cv2.bitwise_or(all_contours_mask, c_mask)
        all_overlap_mask = cv2.bitwise_or(all_overlap_mask, overlap)

    # Save combined masks
    cv2.imwrite(os.path.join(save_dir, "11_all_contours_mask.png"), all_contours_mask)
    cv2.imwrite(os.path.join(save_dir, "12_all_overlap_mask.png"), all_overlap_mask)

    # Overlay visualization
    overlay = image.copy()
    overlay[all_contours_mask == 255] = (0, 0, 255)   # red
    overlay[all_overlap_mask == 255] = (0, 255, 0)    # green

    cv2.imwrite(os.path.join(save_dir, "13_overlap_overlay.png"), overlay)


# ==================================================
# RECURSIVE PROCESSOR
# ==================================================
def process_recursive(input_root, output_root):
    for root, _, files in os.walk(input_root):
        rel_path = os.path.relpath(root, input_root)
        save_base = os.path.join(output_root, rel_path)

        for f in files:
            if not f.lower().endswith(VALID_EXTS):
                continue

            img_path = os.path.join(root, f)
            img = cv2.imread(img_path)

            if img is None:
                print("FAILED:", img_path)
                continue

            name = os.path.splitext(f)[0]
            save_dir = os.path.join(save_base, name)

            tissue_mask = tissue_mask_debug(img, save_dir)
            visualize_overlap(img, tissue_mask, save_dir)

            print("Processed:", img_path)

# ==================================================
# RUN (JUPYTER SAFE)
# ==================================================
process_recursive(PATCH_ROOT, PATCH_OUTPUT_ROOT)
process_recursive(FULL_IMAGE_ROOT, FULL_OUTPUT_ROOT)

print("\nDONE! All nested folders processed correctly.")


Processed: D:\projects\neuro\20x_bf_05_all_patches\1\patch_00000.png
Processed: D:\projects\neuro\20x_bf_05_all_patches\1\patch_00001.png
Processed: D:\projects\neuro\20x_bf_05_all_patches\1\patch_00002.png
Processed: D:\projects\neuro\20x_bf_05_all_patches\1\patch_00003.png
Processed: D:\projects\neuro\20x_bf_05_all_patches\1\patch_00004.png
Processed: D:\projects\neuro\20x_bf_05_all_patches\1\patch_00005.png
Processed: D:\projects\neuro\20x_bf_05_all_patches\1\patch_00006.png
Processed: D:\projects\neuro\20x_bf_05_all_patches\1\patch_00007.png
Processed: D:\projects\neuro\20x_bf_05_all_patches\1\patch_00008.png
Processed: D:\projects\neuro\20x_bf_05_all_patches\2\patch_00000.png
Processed: D:\projects\neuro\20x_bf_05_all_patches\2\patch_00001.png
Processed: D:\projects\neuro\20x_bf_05_all_patches\2\patch_00002.png
Processed: D:\projects\neuro\20x_bf_05_all_patches\2\patch_00003.png
Processed: D:\projects\neuro\20x_bf_05_all_patches\2\patch_00004.png
Processed: D:\projects\neuro\20x_b

In [2]:
import cv2
import numpy as np
import os

# ==================================================
# CONFIG (CHANGE THESE)
# ==================================================
IMAGE_PATH = r"C:\Users\DELL\Downloads\WhatsApp Image 2026-01-24 at 7.16.29 PM.jpeg"
OUTPUT_DIR = r"C:\Users\DELL\Downloads\single_pic"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ==================================================
# TISSUE MASK DEBUG (UNCHANGED LOGIC)
# ==================================================
def tissue_mask_debug(image, save_dir):
    os.makedirs(save_dir, exist_ok=True)

    cv2.imwrite(os.path.join(save_dir, "01_original.png"), image)

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    cv2.imwrite(os.path.join(save_dir, "02_hsv.png"), hsv)

    H, S, V = cv2.split(hsv)
    cv2.imwrite(os.path.join(save_dir, "03_H.png"), H)
    cv2.imwrite(os.path.join(save_dir, "04_S.png"), S)
    cv2.imwrite(os.path.join(save_dir, "05_V.png"), V)

    sat_mask = cv2.inRange(S, 15, 255)
    cv2.imwrite(os.path.join(save_dir, "06_sat_mask.png"), sat_mask)

    not_too_bright = cv2.inRange(V, 0, 250)
    cv2.imwrite(os.path.join(save_dir, "07_not_too_bright.png"), not_too_bright)

    combined = cv2.bitwise_and(sat_mask, not_too_bright)
    cv2.imwrite(os.path.join(save_dir, "08_combined_mask.png"), combined)

    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    opened = cv2.morphologyEx(combined, cv2.MORPH_OPEN, k, iterations=1)
    cv2.imwrite(os.path.join(save_dir, "09_opened.png"), opened)

    closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, k, iterations=2)
    cv2.imwrite(os.path.join(save_dir, "10_closed_final_mask.png"), closed)

    return closed

# ==================================================
# OVERLAP VISUALIZATION
# ==================================================
def visualize_overlap(image, tissue_mask, save_dir):
    os.makedirs(save_dir, exist_ok=True)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    _, fg = cv2.threshold(
        gray, 0, 255,
        cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU
    )

    contours, _ = cv2.findContours(
        fg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    all_contours_mask = np.zeros(tissue_mask.shape, dtype=np.uint8)
    all_overlap_mask = np.zeros(tissue_mask.shape, dtype=np.uint8)

    for c in contours:
        c_mask = np.zeros(tissue_mask.shape, dtype=np.uint8)
        cv2.drawContours(c_mask, [c], -1, 255, -1)

        overlap = cv2.bitwise_and(c_mask, tissue_mask)

        all_contours_mask = cv2.bitwise_or(all_contours_mask, c_mask)
        all_overlap_mask = cv2.bitwise_or(all_overlap_mask, overlap)

    cv2.imwrite(os.path.join(save_dir, "11_all_contours_mask.png"), all_contours_mask)
    cv2.imwrite(os.path.join(save_dir, "12_all_overlap_mask.png"), all_overlap_mask)

    overlay = image.copy()
    overlay[all_contours_mask == 255] = (0, 0, 255)
    overlay[all_overlap_mask == 255] = (0, 255, 0)

    cv2.imwrite(os.path.join(save_dir, "13_overlap_overlay.png"), overlay)

# ==================================================
# RUN FOR SINGLE IMAGE
# ==================================================
image = cv2.imread(IMAGE_PATH)

if image is None:
    raise ValueError("Image not found or path incorrect")

tissue_mask = tissue_mask_debug(image, OUTPUT_DIR)
visualize_overlap(image, tissue_mask, OUTPUT_DIR)

print("DONE! Single image processed successfully.")


DONE! Single image processed successfully.


In [1]:
=======================
    # DEBUG: ACCEPT / REJECT MASKS
    # =======================
    if DEBUG:
        debug_accept = np.zeros(mask_patch.shape, dtype=np.uint8)
        debug_reject = np.zeros(mask_patch.shape, dtype=np.uint8)

    for i, contour in enumerate(contours):

        if hierarchy[i][3] != -1:
            continue

        area = cv2.contourArea(contour)
        if area < MIN_CONTOUR_AREA or area > MAX_CONTOUR_AREA:
            continue

        overlap = contour_overlap_ratio(contour, mask_patch)

        # =======================
        # DEBUG: contour mask
        # =======================
        if DEBUG:
            c_mask = np.zeros(mask_patch.shape, dtype=np.uint8)
            cv2.drawContours(c_mask, [contour], -1, 255, -1)

        if overlap < TISSUE_OVERLAP_MIN:
            if DEBUG:
                debug_reject = cv2.bitwise_or(debug_reject, c_mask)
            continue
        else:
            if DEBUG:
                debug_accept = cv2.bitwise_or(debug_accept, c_mask)

        hull = cv2.convexHull(contour)
        solidity = area / (cv2.contourArea(hull) + 1e-6)
        circ = circularity(contour)
        if solidity < MIN_SOLIDITY or circ < MIN_CIRCULARITY:
            continue

        inner_children = []
        for j, child in enumerate(contours):
            if hierarchy[j][3] == i and cv2.contourArea(child) > 30:
                inner_children.append(child)

        if len(inner_children) == 0:
            continue

        axon_type = "mature" if len(inner_children) <= 2 else "regrowth_cluster"

        (x_outer, y_outer), outer_radius = cv2.minEnclosingCircle(contour)
        contour_shifted = contour + np.array([x_offset, y_offset])
        x_outer += x_offset
        y_outer += y_offset

        outer_area_px = cv2.contourArea(contour)
        outer_area_um2 = outer_area_px * (scale_factor ** 2)

        outer_color = (255, 0, 0) if axon_type == "mature" else (0, 255, 255)
        cv2.drawContours(output_image, [contour_shifted], -1, outer_color, 1)

        inner_radii = []
        inner_areas = []

        for inner_c in inner_children:
            (x_inner, y_inner), inner_radius = cv2.minEnclosingCircle(inner_c)
            inner_shifted = inner_c + np.array([x_offset, y_offset])
            cv2.drawContours(output_image, [inner_shifted], -1, (0, 255, 0), 1)
            inner_radii.append(inner_radius)
            inner_areas.append(cv2.contourArea(inner_c))

        inner_radius = max(inner_radii)
        inner_area_px = max(inner_areas)
        inner_area_um2 = inner_area_px * (scale_factor ** 2)

        thickness = (outer_radius - inner_radius) * scale_factor
        g_ratio = inner_radius / (outer_radius + 1e-6)
        area_ratio = inner_area_um2 / (outer_area_um2 + 1e-6)

        axon_data.append({
            "axon_id": object_counter,
            "axon_type": axon_type,
            "num_inner_contours": len(inner_children),
            "center_x": x_outer,
            "center_y": y_outer,
            "outer_radius": outer_radius * scale_factor,
            "inner_radius": inner_radius * scale_factor,
            "thickness": thickness,
            "diameter": (2 * outer_radius) * scale_factor,
            "outer_area": outer_area_um2,
            "inner_area": inner_area_um2,
            "area_ratio": area_ratio,
            "hole_ratio": inner_area_um2 / (outer_area_um2 + 1e-6),
            "eccentricity": 0,
            "g_ratio": g_ratio
        })

        cv2.putText(
            output_image, str(object_counter),
            (int(x_outer), int(y_outer)),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1
        )

        object_counter += 1

    # =======================
    # DEBUG: SAVE RESULTS
    # =======================
    if DEBUG:
        for name, img in [
            ("debug_accept", debug_accept),
            ("debug_reject", debug_reject)
        ]:
            path = os.path.join(DEBUG_DIR, f"{name}_{y_offset}_{x_offset}.png")
            ok = cv2.imwrite(path, img)
            if not ok:
                print("FAILED to write:", path)

        overlay = patch.copy()
        overlay[debug_accept == 255] = (0, 255, 0)
        overlay[debug_reject == 255] = (0, 0, 255)

        path = os.path.join(DEBUG_DIR, f"debug_overlay_{y_offset}_{x_offset}.png")
        ok = cv2.imwrite(path, overlay)
        if not ok:
            print("FAILED to write:", path)

    return object_counter

# --------------------------
# Whole image processing
# --------------------------
def process_image(image_path, debug_dir, patch_size=1024):
    image = cv2.imread(image_path)
    if image is None:
        print(f" Cannot read image: {image_path}")
        return None, None

    tissue_mask_full = build_tissue_mask(image)
    h, w = image.shape[:2]

    output_image = image.copy()
    axon_data = []
    object_counter = 1

    # =======================
    # ADD: FULL-IMAGE DEBUG BUFFERS
    # =======================
    debug_full_cleaned = np.zeros((h, w), dtype=np.uint8)
    debug_full_accept  = np.zeros((h, w), dtype=np.uint8)
    debug_full_reject  = np.zeros((h, w), dtype=np.uint8)
    debug_full_overlay = image.copy()

    for y in range(0, h, patch_size):
        for x in range(0, w, patch_size):
            patch = image[y:y+patch_size, x:x+patch_size]
            mask_patch = tissue_mask_full[y:y+patch_size, x:x+patch_size]

            object_counter = process_patch(
                patch, mask_patch, x, y,
                axon_data, object_counter,
                output_image,
                debug_dir
            )
            
            # ===== STITCH DEBUG PATCH INTO FULL IMAGE =====
            dbg_dir = os.path.join(debug_dir, "debug_outputs")
            
            for name in ["debug_cleaned", "debug_accept", "debug_reject", "debug_overlay"]:
                p = os.path.join(dbg_dir, f"{name}_{y}_{x}.png")
                if not os.path.exists(p):
                    continue
            
                patch_dbg = cv2.imread(p, 0 if name != "debug_overlay" else 1)
                ph, pw = patch_dbg.shape[:2]
            
                if name == "debug_cleaned":
                    debug_full_cleaned[y:y+ph, x:x+pw] = patch_dbg
            
                elif name == "debug_accept":
                    debug_full_accept[y:y+ph, x:x+pw] |= patch_dbg
            
                elif name == "debug_reject":
                    debug_full_reject[y:y+ph, x:x+pw] |= patch_dbg
            
                elif name == "debug_overlay":
                    debug_full_overlay[y:y+ph, x:x+pw] = patch_dbg

    # =======================
    # ADD: SAVE STITCHED DEBUG IMAGES
    # =======================
    cv2.imwrite(os.path.join(debug_dir, "debug_full_cleaned.png"), debug_full_cleaned)
    cv2.imwrite(os.path.join(debug_dir, "debug_full_accept.png"), debug_full_accept)
    cv2.imwrite(os.path.join(debug_dir, "debug_full_reject.png"), debug_full_reject)
    cv2.imwrite(os.path.join(debug_dir, "debug_full_overlay.png"), debug_full_overlay)

    print(f"Processed {len(axon_data)} axons.")
    return axon_data, output_image


# --------------------------
# Save CSV
# --------------------------
def save_to_csv(axon_data, output_folder):
    df = pd.DataFrame(axon_data)
    csv_path = os.path.join(output_folder, "axon_measurements.csv")
    df.to_csv(csv_path, index=False)
    return df

# --------------------------
# Visualizations
# --------------------------
def plot_and_visualize(df, output_folder):
    numeric_cols = df.select_dtypes(include=[np.number]).columns

    for col in numeric_cols:
        vals = df[col].dropna()
        if len(vals) == 0:
            continue

        plt.hist(vals, bins=25)
        plt.title(col)
        plt.savefig(os.path.join(output_folder, f"hist_{col}.png"))
        plt.close()

# --------------------------
# Process entire folder
# --------------------------
def process_folder(input_folder, output_root):
    images = [
        f for f in os.listdir(input_folder)
        if f.lower().endswith((".png", ".jpg", ".jpeg", ".bmp"))
    ]

    for img_name in images:
        print("\n==============================")
        print(f"Processing {img_name}")
        print("==============================")

        img_path = os.path.join(input_folder, img_name)
        img_name_wo_ext = os.path.splitext(img_name)[0]
        img_output_folder = os.path.join(output_root, img_name_wo_ext)
        os.makedirs(img_output_folder, exist_ok=True)

        axon_data, output_image = process_image(img_path,img_output_folder)

        if axon_data is None:
            continue

        df = save_to_csv(axon_data, img_output_folder)
        plot_and_visualize(df, img_output_folder)
        image = cv2.imread(img_path)
        tissue_mask = build_tissue_mask(image)
        
        mask_path = os.path.join(img_output_folder, "tissue_mask.png")
        cv2.imwrite(mask_path, tissue_mask)

        out_img_path = os.path.join(img_output_folder, f"numbered_{img_name}")
        cv2.imwrite(out_img_path, output_image)

        print(f" Saved results for {img_name} in {img_output_folder}\n")

def process_multiple_folders(input_root, output_root):
    subfolders = [
        f for f in os.listdir(input_root)
        if os.path.isdir(os.path.join(input_root, f))
    ]

    print(f"Found {len(subfolders)} folders:", subfolders)

    for folder_name in subfolders:
        input_subfolder = os.path.join(input_root, folder_name)
        output_subfolder = os.path.join(output_root, folder_name)

        os.makedirs(output_subfolder, exist_ok=True)

        print("\n====================================")
        print(f"Processing folder: {folder_name}")
        print("====================================")

        process_folder(input_subfolder, output_subfolder)

    print("\n ALL FOLDERS PROCESSED!")


# --------------------------
# RUN
# --------------------------
input_root = r"D:\projects\neuro\20x_bf_05_all_patches"
output_root = r"D:\projects\neuro\20x_bf_05_all_debug"

process_multiple_folders(input_root, output_root)


print("\n DONE! All images processed.")


Found 5 folders: ['1', '2', '3', '4', '5']

Processing folder: 1

Processing patch_00000.png
Processed 1 axons.
 Saved results for patch_00000.png in D:\projects\neuro\20x_bf_05_all_debug\1\patch_00000


Processing patch_00001.png
Processed 17 axons.
 Saved results for patch_00001.png in D:\projects\neuro\20x_bf_05_all_debug\1\patch_00001


Processing patch_00002.png
Processed 15 axons.
 Saved results for patch_00002.png in D:\projects\neuro\20x_bf_05_all_debug\1\patch_00002


Processing patch_00003.png
Processed 11 axons.
 Saved results for patch_00003.png in D:\projects\neuro\20x_bf_05_all_debug\1\patch_00003


Processing patch_00004.png
Processed 29 axons.
 Saved results for patch_00004.png in D:\projects\neuro\20x_bf_05_all_debug\1\patch_00004


Processing patch_00005.png
Processed 19 axons.
 Saved results for patch_00005.png in D:\projects\neuro\20x_bf_05_all_debug\1\patch_00005


Processing patch_00006.png
Processed 20 axons.
 Saved results for patch_00006.png in D:\projects\neur